In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
import pickle
import random
from tqdm import tqdm
from PIL import Image
from sklearn.metrics import accuracy_score
import numpy as np
import os

In [2]:
class SiameseSignatureDataset(Dataset):
    def __init__(self, pairs):
        self.pairs = pairs
        self.transform = transforms.Compose([
            transforms.Grayscale(num_output_channels=3),  
            transforms.ToTensor(),                        
        ])
    def __len__(self):
        return len(self.pairs)
    def __getitem__(self, idx):
        img1, img2, label = self.pairs[idx]
        img1 = self.transform(img1)
        img2 = self.transform(img2)
        label = torch.tensor(label, dtype=torch.float32)
        return img1, img2, label

In [3]:
class SiameseResNet(nn.Module):
    def __init__(self):
        super(SiameseResNet, self).__init__()
        base_model = models.resnet18(weights=None)
        base_model.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        num_ftrs = base_model.fc.in_features
        base_model.fc = nn.Identity()
        self.base_model = base_model
        self.embedding = nn.Sequential(
            nn.Linear(num_ftrs, 256),
            nn.ReLU(),
            nn.Linear(256, 128)
        )
    def forward_once(self, x):
        x = self.base_model(x)
        x = self.embedding(x)
        return x
    def forward(self, img1, img2):
        out1 = self.forward_once(img1)
        out2 = self.forward_once(img2)
        return out1, out2

In [4]:
class ContrastiveLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin
    def forward(self, out1, out2, label):
        euclidean_distance = F.pairwise_distance(out1, out2)
        loss = label * torch.pow(euclidean_distance, 2) + \
               (1 - label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2)
        return torch.mean(loss)

In [5]:
with open("C:/Active-Repositories/secure-signature-verification/image_pairs/positive_pairs.pkl", "rb") as f1:
    positive_pairs = pickle.load(f1)
negative_pairs = []
for i in range(1, 3):
    with open(f"C:/Active-Repositories/secure-signature-verification/image_pairs/negative_pairs_part{i}.pkl", "rb") as f:
        negative_pairs.extend(pickle.load(f))
all_pairs = positive_pairs + negative_pairs
random.shuffle(all_pairs)
split = int(0.8 * len(all_pairs))
train_pairs = all_pairs[:split]
test_pairs = all_pairs[split:]
train_dataset = SiameseSignatureDataset(train_pairs)
test_dataset = SiameseSignatureDataset(test_pairs)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [6]:
print(len(positive_pairs))
print(len(negative_pairs))

36000
38400


In [ ]:
os.makedirs("checkpoints", exist_ok=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SiameseResNet().to(device)
criterion = ContrastiveLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
epochs = 5
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for img1, img2, label in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        img1, img2, label = img1.to(device), img2.to(device), label.to(device)
        out1, out2 = model(img1, img2)
        loss = criterion(out1, out2, label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}")
    checkpoint_path = f"checkpoints/siamese_epoch_{epoch+1}.pth"
    torch.save(model.state_dict(), checkpoint_path)


Epoch 1: 100%|██████████| 1860/1860 [3:00:25<00:00,  5.82s/it]  


Epoch [1/5], Loss: 0.0942


Epoch 2: 100%|██████████| 1860/1860 [2:46:11<00:00,  5.36s/it]  


Epoch [2/5], Loss: 0.0177


Epoch 3: 100%|██████████| 1860/1860 [2:40:32<00:00,  5.18s/it]  


Epoch [3/5], Loss: 0.0083


Epoch 4: 100%|██████████| 1860/1860 [2:39:40<00:00,  5.15s/it]  


Epoch [4/5], Loss: 0.0065


Epoch 5: 100%|██████████| 1860/1860 [4:15:50<00:00,  8.25s/it]     

Epoch [5/5], Loss: 0.0052


In [8]:
torch.save(model.state_dict(), "siamese_resnet_18.pth")

In [9]:
val_split = int(0.9 * len(train_pairs))
val_pairs = train_pairs[val_split:]
train_pairs = train_pairs[:val_split]
val_dataset = SiameseSignatureDataset(val_pairs)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
all_distances = []
all_labels = []
with torch.no_grad():
    for img1, img2, labels in val_loader:
        img1, img2, labels = img1.to(device), img2.to(device), labels.to(device)
        out1, out2 = model(img1, img2)
        distances = F.pairwise_distance(out1, out2)
        all_distances.extend(distances.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
thresholds = np.linspace(0, max(all_distances), 200)
best_acc = 0
best_threshold = 0
for t in thresholds:
    preds = (np.array(all_distances) < t).astype(float)
    acc = accuracy_score(all_labels, preds)
    if acc > best_acc:
        best_acc = acc
        best_threshold = t
print(f"Best validation threshold: {best_threshold:.4f} with accuracy: {best_acc:.2%}")

Best validation threshold: 0.4275 with accuracy: 100.00%
